In [163]:
import torch
X = torch.randn(5000 , 764)
y = torch.randint(0, 9, (5000,))

import torch.nn.functional as F

# class Module():
#     def forward(self,*args):
#         raise NotImplementedError("Forward Method not implemented")
#     def __call__(self,*args):
#         self.args = args
#         self.out = self.forward(*args)
#         return self.out
#     def backward(self): self.bwd(self.out, *self.args)
#     def bwd(self,*args): raise Exception('not implemented') 

    
class Module():
    def __call__(self, *args):
        # import pdb ; pdb.set_trace()
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')
    
    
class Linear(Module):
    def __init__(self, ni , no):
        super().__init__()
        self.w = torch.randn(ni, no).requires_grad_()
        self.b = torch.zeros(no).requires_grad_()
    def forward(self,x):
        return x@self.w + self.b
    def bwd(self , out , inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)


class Relu(Module):
    def __init__(self):
        pass
    
    def forward(self,x):
        self.out = x.clamp_min(0.)
        return self.out
    
    def bwd(self,out,inp):
        inp.g = (inp>0).float() * out.g
        


class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]
                
        
        

class Model(Module):
    def __init__(self , num_hidden_layers , ni  , nh , no):
        
        
        self.relu = Relu()
        
        # Stacking layers
        self.l1 = [ Linear(ni,nh) , self.relu ]
        self.hidden_layers = []
        for _ in range(num_hidden_layers-1):
            self.hidden_layers.append(Linear(nh,nh))
            self.hidden_layers.append(self.relu)
        self.out_layer = [Linear(nh,no)]
        
        self.layers = self.l1 + self.hidden_layers + self.out_layer
        self.mse = Mse() 
        
        
    def forward(self,x):    
        
        for layer in self.layers:
            # import pdb ; pdb.set_trace()
            x = layer(x)
        return x
    
    def __call__(self,x, y):
        x = self.forward(x)
        self.loss = self.mse(x , y)
        return self.loss
    
    def backward(self,*args):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()
    
            
                                     
    
ni = X.shape[1]
nh = 32
no = 1
num_hidden_layers = 1
model = Model(num_hidden_layers,ni,nh,no)
loss = model(X,y)
model.backward()





AttributeError: 'Tensor' object has no attribute 'g'

AttributeError: 'Tensor' object has no attribute 'g'

In [159]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

class Lin(Module):
    def __init__(self, w, b): 
        self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()


In [160]:
w1 = torch.randn(ni,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [162]:
model = Model(w1, b1, w2, b2)

loss = model(X, y)

model.backward()

In [156]:
def tes(*args):
    arguments = args
    
x = torch.tensor(9)
tes(x)
print(x , x.g)

AttributeError: 'Tensor' object has no attribute 'g'